### AVALIAÇÃO FINAL   
### MÓDULO EXPLORAÇÃO DE DADOS NA INTERNET

### Objetivo
####Raspar os dados de uma livraria online: http://books.toscrape.com/
#### Extrair apenas duas informações de cada livro: 
 1. Título 
 2. Descrição.

####Bibliotecas

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

####Função para solicitar e analisar a página Web

In [2]:
url = "http://books.toscrape.com/index.html"

def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

print(getAndParseURL(url).prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

####Função para buscar links de livros em qualquer página do site

In [3]:
def getBooksURLs(url):
    soup = getAndParseURL(url)
    
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href')
       for x in soup.findAll("article", class_ = "product_pod")])

####Obter todas as páginas URLs

In [4]:
pages_urls = [url]

soup = getAndParseURL(pages_urls[0])

while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    pages_urls.append(new_url)
    soup = getAndParseURL(new_url)
    
print(str(len(pages_urls)) + " fetched URLs")
print("Some examples:")
pages_urls[:5]

50 fetched URLs
Some examples:


['http://books.toscrape.com/index.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

####Busca todos os URLs dos produtos para cada página

In [5]:
booksURLs = []

for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " fetched URLs")
print("Some examples:")
booksURLs[:5]

1000 fetched URLs
Some examples:


['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html']

####Busca os dados de cada produto

In [6]:
title = []
description = []

for url in booksURLs:
    soup = getAndParseURL(url)
    # product title
    title.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    # product description
    description.append(soup.find_all('p')[3].get_text())

####Armazena dentro de um DataFrame do Pandas.

In [7]:
df_scraped = pd.DataFrame({'Title': title, 'Description': description})

df_scraped

,Title,Description
0,A Light in the Attic,It's hard to imagine a world without A Light i...
1,Tipping the Velvet,"""Erotic and absorbing...Written with starling ..."
2,Soumission,"Dans une France assez proche de la nÃ´tre, un ..."
3,Sharp Objects,"WICKED above her hipbone, GIRL across her hear..."
4,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...
...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,\n\n\n\n\n\n
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",High school student Kei Nagai is struck dead i...
997,A Spy's Devotion (The Regency Spies of London #1),"In Englandâs Regency era, manners and elegan..."
998,1st to Die (Women's Murder Club #1),"James Patterson, bestselling author of the Ale..."
